## 1. Importing Libraries and Loading data into pandas  Dataframe

In [5]:
import pandas as pd

data = [
    {"user_id": 1, "destination": "Lalibela",       "rating": 5},
    {"user_id": 1, "destination": "Axum",           "rating": 4},
    {"user_id": 2, "destination": "Lalibela",       "rating": 4},
    {"user_id": 2, "destination": "Gonder",         "rating": 5},
    {"user_id": 3, "destination": "Axum",           "rating": 5},
    {"user_id": 3, "destination": "Hawassa",        "rating": 3},
    {"user_id": 4, "destination": "Gonder",         "rating": 4},
    {"user_id": 4, "destination": "Lalibela",       "rating": 3},
    {"user_id": 5, "destination": "Arba Minch",     "rating": 5},
    {"user_id": 5, "destination": "Hawassa",        "rating": 4},
]

df = pd.DataFrame(data)
print(df)


   user_id destination  rating
0        1    Lalibela       5
1        1        Axum       4
2        2    Lalibela       4
3        2      Gonder       5
4        3        Axum       5
5        3     Hawassa       3
6        4      Gonder       4
7        4    Lalibela       3
8        5  Arba Minch       5
9        5     Hawassa       4


## 2. Build User–Item Matrix

This matrix is the core of collaborative filtering.

In [6]:
user_item_matrix = df.pivot_table(index="user_id", columns="destination", values="rating").fillna(0)

print("\nUser-Item Matrix:\n", user_item_matrix)



User-Item Matrix:
 destination  Arba Minch  Axum  Gonder  Hawassa  Lalibela
user_id                                                 
1                   0.0   4.0     0.0      0.0       5.0
2                   0.0   0.0     5.0      0.0       4.0
3                   0.0   5.0     0.0      3.0       0.0
4                   0.0   0.0     4.0      0.0       3.0
5                   5.0   0.0     0.0      4.0       0.0


## 3. Compute User Similarity

We find which users have similar preferences using cosine similarity.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(user_item_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

print("\nUser Similarity Matrix:\n", similarity_df)



User Similarity Matrix:
 user_id         1         2         3         4         5
user_id                                                  
1        1.000000  0.487805  0.535672  0.468521  0.000000
2        0.487805  1.000000  0.000000  0.999512  0.000000
3        0.535672  0.000000  1.000000  0.000000  0.321403
4        0.468521  0.999512  0.000000  1.000000  0.000000
5        0.000000  0.000000  0.321403  0.000000  1.000000


## 4. Recommendation Function

Algorithm:

    1.Find similar users

    2.Aggregate their ratings

    3. Recommend destinations that the user has not rated ye

In [8]:
def recommend_for_user(target_user, top_n=3):

    # Step 1: similar users
    user_similarities = similarity_df[target_user].sort_values(ascending=False)

    # Step 2: ratings from other users weighted by similarity
    weighted_ratings = user_item_matrix.T.dot(user_similarities)

    # Step 3: convert to DataFrame
    weighted_ratings = weighted_ratings.to_frame(name="score")

    # remove destinations the user already rated
    already_rated = user_item_matrix.loc[target_user]
    weighted_ratings = weighted_ratings[already_rated == 0]

    # return top N results
    return weighted_ratings.sort_values("score", ascending=False).head(top_n)


## 5. Test Recommendation

Let’s get recommendations by user id.

In [18]:
recommendations = recommend_for_user( 4)
print("\nRecommended for User id :\n", recommendations)



Recommended for User id :
                 score
destination          
Axum         1.874085
Arba Minch   0.000000
Hawassa      0.000000
